In [0]:
%run "../utils/mount_configuration"

In [0]:
evaluation_date = dbutils.jobs.taskValues.get(taskKey="tickers_quality_evaluation", key="evaluation_date", debugValue="2025-03-17")

In [0]:
tickers_dividend_df = spark.read.format('delta') \
                                 .load(f'{processed_folder_path}/tickers_dividend')

display(tickers_dividend_df)

In [0]:
from pyspark.sql.functions import col

warren_buffet_stocks_selection_df = spark.read.format('delta') \
                                              .load(f'{presentation_folder_path}/warren_buffet_stocks_selection') \
                                              .filter(col('date') == evaluation_date) \
                                              .select('ticker') \
                                              .distinct()

display(warren_buffet_stocks_selection_df)

In [0]:
benjamin_graham_stocks_selection = spark.read.format('delta') \
                                              .load(f'{presentation_folder_path}/benjamin_graham_stocks_selection') \
                                              .filter(col('date') == evaluation_date) \
                                              .select('ticker') \
                                              .distinct()

display(benjamin_graham_stocks_selection)

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import col, sum, year
from datetime import datetime


def save_plot_to_data_lake(data, x_col, y_col, hue_col, title, xlabel, ylabel, filename):
    plt.figure(figsize=(12, 6))
    sns.barplot(x=x_col, y=y_col, hue=hue_col, data=data, palette="coolwarm")
    plt.title(title, fontsize=16)
    plt.xlabel(xlabel, fontsize=14)
    plt.ylabel(ylabel, fontsize=14)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.grid(True, axis="y", linestyle="--", alpha=0.7)
    plt.gcf().set_facecolor('white')
    plt.savefig(f'/dbfs/{presentation_folder_path}/investment_recommendations/{evaluation_date}/{filename}.png')
    plt.close()


tickers_to_plot = [row['ticker'] for row in benjamin_graham_stocks_selection.select("ticker").distinct().collect()]

#Creation directory in which will be stored all files for each evalution date 
directory = f'{presentation_folder_path}/investment_recommendations/{evaluation_date}'
dbutils.fs.mkdirs(directory)

#Creation dataframe for last two years of dividends
last_2_years_dividends_df = tickers_dividend_df \
    .filter(col("pay_date") >= datetime(datetime.now().year - 2, 1, 1)) \
    .filter(col("ticker").isin(tickers_to_plot)) \
    .groupBy("ticker") \
    .agg(sum("cash_amount").alias("total_dividends")) \
    .toPandas()

#Creation dataframe for the last 10 years of dividends per year for selected tickers
df_last_10_years_dividends_per_year = tickers_dividend_df \
    .filter(col("pay_date") >= datetime(datetime.now().year - 10, 1, 1)) \
    .withColumn("year", year(col("pay_date"))) \
    .groupBy("ticker", "year") \
    .agg(sum("cash_amount").alias("total_dividends_per_year")) \
    .filter(col("ticker").isin(tickers_to_plot)) \
    .toPandas()


save_plot_to_data_lake(last_2_years_dividends_df, "ticker", "total_dividends", None, 
                       "Sum of Dividends from the Last Two Years for Selected Tickers",  "Ticker", "Total Dividends per Share",
                       "dividends_last_2_years_plot_benjamin_graham")

save_plot_to_data_lake(df_last_10_years_dividends_per_year, "year", "total_dividends_per_year", "ticker",
                       "Dividends Each Year for the Last 10 Years for Selected Tickers", "Year", "Sum of Dividends per Share for Each Year",
                       "dividends_per_year_plot_benjamin_graham")



In [0]:
dbutils.fs.ls(f'{presentation_folder_path}/investment_recommendations/{evaluation_date}/')